In [10]:
#Import or collect tweets using a keyword or hashtag. Watch your rate limiting
# After setting up a twitter account and applying for a developer account, my approval still has not come.
#This may be because I am not a twitter user and just opened the account and have not done anything with it other than the application?
#I can no longer wait for approval so I found a tweet based file in Kaggle that contains geographic data and will download it
# into the appropriate file and start from there.

import pandas as pd
df = pd.read_csv('tweets.csv')
print(df.dtypes)




id           int64
keyword     object
location    object
text        object
target       int64
dtype: object


In [11]:
#Use Pandas to sort and analyze your tweets.
#Prepping the data to graph keyword and location.
df = df.loc[df['location'].notnull()]

print(df[['keyword','location']].head())

  keyword         location
2  ablaze    New York City
3  ablaze   Morgantown, WV
5  ablaze               OC
6  ablaze  London, England
7  ablaze           Bharat


In [12]:
#Use Pandas to sort and analyze your tweets.
locs = df['location'].value_counts()
print(locs)

EVERYWHERE                 3
United Kingdom             3
Melbourne, Victoria        3
Ottawa, Ontario, Canada    3
London                     3
                          ..
Texas, USA                 1
St Augustine, FL           1
RVA✈️FLA                   1
PA                         1
Tempe, AZ                  1
Name: location, Length: 197, dtype: int64


In [13]:
#Calculate user coordinates, where feasible, for the set of collected tweets.
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='twitter-analysis-client')
limited = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def find_location(row):
    place = row['location']
    location = limited(place)
    
    if location != None:
        return location.latitude, location.longitude
    else:
        return "Not Found", "Not Found"

df[['latitude','longitude']] = df.apply(find_location, axis="columns", result_type="expand")
df

,id,keyword,location,text,target,latitude,longitude
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,40.712728,-74.006015
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,39.629681,-79.955944
5,5,ablaze,OC,"If this child was Chinese, this tweet would ha...",0,48.353559,27.395434
6,6,ablaze,"London, England",Several houses have been set ablaze in Ngemsib...,1,51.507322,-0.127647
7,7,ablaze,Bharat,Asansol: A BJP office in Salanpur village was ...,1,22.351115,78.667743
...,...,...,...,...,...,...,...
316,316,annihilation,"Baltimore, MD",Are you speaking about the atrocities and anni...,1,39.290882,-76.610759
317,317,annihilation,India,Yeah I was surprised too when I heard it first...,0,22.351115,78.667743
318,318,annihilation,"Mumbai, India","You don't understand systemic oppression, that...",0,19.07599,72.877393
319,319,annihilation,Island,"I dont get your bias flipping, short annihilat...",0,64.984182,-18.105901
